Стандартная версия Нужно реализовать rest api на базе flask в google colab.

выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
Реализовать ноутбук с сервером
Реализовать ноутбук с клиентом
Сложная версия

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта)
итоговый проект должен содержать:
каталог app/models/ (здесь модель-пайплайн предобученная либо код обучения модели-пайплайна)
файл app/run_server.py (здесь основной код flask-приложения)
requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта)
README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д)
*Dockerfile
*docker-entrypoint.sh
*front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

подопытный- все тот же набор диабетиков из седьмого

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets


%matplotlib inline

In [2]:
df = pd.read_csv('C:/Users/u187s/LEARNING/MLB/9/diabetes_prediction_dataset.csv')

In [3]:
df.head(10)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
5,Female,20.0,0,0,never,27.32,6.6,85,0
6,Female,44.0,0,0,never,19.31,6.5,200,1
7,Female,79.0,0,0,No Info,23.86,5.7,85,0
8,Male,42.0,0,0,never,33.64,4.8,145,0
9,Female,32.0,0,0,never,27.32,5.0,100,0


как видим, датасет смешанный

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [5]:
df.describe()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [6]:
df.isnull()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
99995,False,False,False,False,False,False,False,False,False
99996,False,False,False,False,False,False,False,False,False
99997,False,False,False,False,False,False,False,False,False
99998,False,False,False,False,False,False,False,False,False


посмотрим на соотношение в целевой переменной (здоровых значительно больше, дисбаланс по целевой переменной более 1/10)

In [7]:
df['diabetes'].value_counts()

0    91500
1     8500
Name: diabetes, dtype: int64

для финалочки посмотреть на уникальные значения, и можно делать вывод о характере датасета

In [8]:
for col in df:
    print("уникальные значения в столбце", col, ":", df[col].unique())

уникальные значения в столбце gender : ['Female' 'Male' 'Other']
уникальные значения в столбце age : [80.   54.   28.   36.   76.   20.   44.   79.   42.   32.   53.   78.
 67.   15.   37.   40.    5.   69.   72.    4.   30.   45.   43.   50.
 41.   26.   34.   73.   77.   66.   29.   60.   38.    3.   57.   74.
 19.   46.   21.   59.   27.   13.   56.    2.    7.   11.    6.   55.
  9.   62.   47.   12.   68.   75.   22.   58.   18.   24.   17.   25.
  0.08 33.   16.   61.   31.    8.   49.   39.   65.   14.   70.    0.56
 48.   51.   71.    0.88 64.   63.   52.    0.16 10.   35.   23.    0.64
  1.16  1.64  0.72  1.88  1.32  0.8   1.24  1.    1.8   0.48  1.56  1.08
  0.24  1.4   0.4   0.32  1.72  1.48]
уникальные значения в столбце hypertension : [0 1]
уникальные значения в столбце heart_disease : [1 0]
уникальные значения в столбце smoking_history : ['never' 'No Info' 'current' 'former' 'ever' 'not current']
уникальные значения в столбце bmi : [25.19 27.32 23.45 ... 59.42 44.39 60.52

В общем и целом уже понятно, что датасет:
- смешанный
- чистый, без пропусков, nan и т.п. мусора
- с определенным, местами огромным дисбалансом классов.
Что касается возможных "выбросов"- не думаю, что в медицинском датасете они вызваны ошибками или ими можно пренебречь: лучше залечить здорового.

что касается алгоритма ML, который сегодня выберем: на этот раз LogRegression

разбиваем датасет на выборки, рандомно, с сохранением пропорций по таргету (stratify) и сохраняем на диск:

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df, df['diabetes'], random_state=42, stratify=df['diabetes'])
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)


In [10]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class MissingValueImputer(BaseEstimator, TransformerMixin):
    """
    Transformer to fill missing values with a specified value.
    """

    def __init__(self, key, value):
        self.key = key
        self.value = value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if X[self.key].dtype == 'object':
            return X
        else:
            mean_value = X[self.key].mean()
            X.loc[:, self.key] = X.loc[:, self.key].fillna(self.value)
            return X

In [11]:
features = ['age', 'bmi', 'blood_glucose_level']
target = ['diabetes']

In [12]:
# combine
age = Pipeline([
                ('imputer', MissingValueImputer('age','')),
                ('selector', ColumnSelector(key='age'))
            ])

bmi = Pipeline([
                ('imputer', MissingValueImputer('bmi','')),
                ('selector', ColumnSelector(key='bmi'))
            ])

blood_glucose_level = Pipeline([
                ('imputer', MissingValueImputer('blood_glucose_level','')),
                ('selector', ColumnSelector(key='blood_glucose_level'))
            ])


feats = FeatureUnion([
                ('age', age),
                ('bmi', bmi),
                ('blood_glucose_level', blood_glucose_level)
            ])

In [13]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

Wall time: 149 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('imputer',
                                                                  MissingValueImputer(key='age',
                                                                                      value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='age'))])),
                                                ('bmi',
                                                 Pipeline(steps=[('imputer',
                                                                  MissingValueImputer(key='bmi',
                                                                                      value='')),
                                                                 ('selector',
                                                              

In [14]:
# Сохранение пайплайна
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [15]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [16]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [17]:
preds[:20]

array([0.00300356, 0.00608521, 0.00029373, 0.02097869, 0.14735215,
       0.06320181, 0.02634697, 0.01502631, 0.00352593, 0.16040044,
       0.0270516 , 0.0024589 , 0.01638641, 0.00770431, 0.26095808,
       0.12322615, 0.01475117, 0.02380036, 0.00492399, 0.001976  ])

In [18]:
pred_df.head(5)

,preds
0,0.003004
1,0.006085
2,0.000294
3,0.020979
4,0.147352


FLASK

# **При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование.

In [19]:
#!pip install flask-ngrok

In [20]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [21]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [22]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    
    age, bmi, blood_glucose_level = "", "", ""
    request_json = request.get_json()

    if request_json["age"]:
        age = request_json['age']

    if request_json["bmi"]:
        bmi = request_json['bmi']

    if request_json["blood_glucose_level"]:
        blood_glucose_level = request_json['blood_glucose_level']

    #print(description)
    preds = model.predict_proba(pd.DataFrame({"age": [age],
                                              "bmi": [bmi],
                                              "blood_glucose_level": [blood_glucose_level]}))
    data["predictions"] = preds[:, 1][0]
    data["description"] = description
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3fc0-79-165-149-13.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Jul/2023 19:25:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 19:25:16] "GET /static/EuclidSquare-Regular-WebS.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:16] "GET /static/EuclidSquare-Medium-WebS.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:16] "GET /static/IBMPlexMono-TextItalic.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:16] "GET /static/IBMPlexMono-Text.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:16] "GET /static/IBMPlexMono-SemiBoldItalic.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:16] "GET /static/IBMPlexMono-SemiBold.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 19:25:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:17] "GET /static/EuclidSquare-MediumItalic-WebS.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:17] "GET /static/EuclidSquare-RegularItalic-WebS.woff HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2023 19:25:18] "G